In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from datetime import timedelta
from sklearn.grid_search import GridSearchCV

import sys
sys.path.append('/Users/drewrice/Desktop/Github/Optimizing-Malpractice-Insurance/code')
from return_low_no_suit_score import return_low_no_suit_score
from return_probability_scores import return_probability_scores

# Imports and feature selection

In [7]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds_full = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')
policy = pd.read_csv('/Users/drewrice/Desktop/capson_csv/PolicyInfo.csv')

In [8]:
docs.shape

(7252, 14)

In [9]:
# drop non-essential columns
# drop duplicates
docs.drop(['Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [10]:
# grab essential columns for building classifier
indem = claims[['Indemnity_Paid','ProviderId']]

In [12]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','Middle Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','Middle Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [13]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [14]:
# def build_classifier_label(item):
#     if item == 0:
#         return 'suit, no indem'
#     if item != 0:
#         return 'suit, indem'

def build_classifier_label(item):
    if item == 0:
        return 1
    if item != 0:
        return 2

In [15]:
indem['indem_label'] = indem['Indemnity_Paid'].apply(lambda item: build_classifier_label(item))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Join docs and indemnity tables, fill NaNs with 'no suit'

In [18]:
joined = pd.merge(docs, indem[['ProviderId', 'indem_label']], how='left', on='ProviderId')
joined.drop_duplicates(inplace=True)

In [19]:
# finish indemnity classifier
# find NaNs in indem_label column, replace with 'no suit'
def indem_label_no_suit(x):
    if type(x) == np.float_: 
        if np.isnan(x): 
#             return 'no suit'
            return 0
        else:
            return x

In [20]:
joined.indem_label = joined.indem_label.apply(lambda item: indem_label_no_suit(item))

# Claims data
#### building classifiers for BoardCredit and ClaimsBand, then merging with joined

In [431]:
def build_classifier(item):
    if item == 0:
        return 0
    if item < 0:
        return 1
    if item > 0:
        return 2

In [432]:
# resave debs_and_creds with just the desired columns
# NOTE: typo 'RecordKepping'
debs_and_creds = debs_and_creds_full[['ProviderId','BoardCredit','ClaimsBand','Procedures','JCAHO','RecordKepping']]

# lower Provider Id
debs_and_creds['ProviderId'] = debs_and_creds['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())
debs_and_creds.tail()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ProviderId,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping
8458,19d9c4e6-e820-41c4-b398-134e6c0ba410,-55,-220,0,0,0
8459,6398eeea-10af-448f-bd10-364be1d0c90c,0,-8951,0,-2984,0
8460,d54c0b59-08e5-4ed7-ba5a-413b2016b269,-1366,0,0,-1366,-2733
8461,3eacedf6-9173-4a64-89c7-ba8b6e150850,-88,0,0,0,0
8462,ea83f592-67dd-46d5-b15c-9d0738f81c93,0,-258,0,0,0


In [433]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()
# reset index required
debs_and_creds.reset_index(inplace=True)

In [434]:
# use build_classifier to create binary column from BoardCredit & ClaimsBand
debs_and_creds['BoardCredit'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['ClaimsBand'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))
debs_and_creds['Procedures'] = debs_and_creds['Procedures'].apply(lambda item: build_classifier(item))
debs_and_creds['JCAHO'] = debs_and_creds['JCAHO'].apply(lambda item: build_classifier(item))
debs_and_creds['RecordKepping'] = debs_and_creds['RecordKepping'].apply(lambda item: build_classifier(item))

# Build 'joined'

In [435]:
joined = pd.merge(joined, debs_and_creds, how='left', on='ProviderId')

# EDA

In [436]:
# print num of doctors in each bucket, using the 
print 'Doctors wih no suits: ', joined[joined['indem_label'] == 0].sort(['Last Name']).shape[0]
print 'Doctors with suits, no indemnity paid: ', joined[joined['indem_label'] == 1].sort(['Last Name']).shape[0]
print 'Doctors with suits, indemnity paid: ', joined[joined['indem_label'] == 2].sort(['Last Name']).shape[0]

Doctors wih no suits:  6147
Doctors with suits, no indemnity paid:  281
Doctors with suits, indemnity paid:  41


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# Feature engineering

### ~ build age column ~

In [437]:
# convert DoB to datetime object
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [438]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [439]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [440]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [441]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [442]:
# function to convert years into days
def to_years(dt):
    if dt:
        dt = (dt.days) / 365.25
    return dt

In [443]:
# apply to_years function to age column
joined['age'] = (joined['age'].apply(lambda dt: to_years(dt))).round()

In [444]:
# refill Date of Birth with NaNs
joined['Date of Birth'].replace(datetime(2060, 1, 1, 0, 0), np.nan, inplace=True)

In [445]:
def refill_age(x):
    if x < 0:
        x = np.nan
    return x

In [446]:
# refill age with nan's for docs with missing birthdate
joined['age'] = (joined['age'].apply(lambda dt: refill_age(dt))).round()

In [447]:
joined.corr()

,indem_label,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping,age
indem_label,1.000000,0.131730,0.114054,0.166064,0.191448,0.047917,0.113401
BoardCredit,0.131730,1.000000,0.525431,0.036848,0.434245,0.169449,0.109200
ClaimsBand,0.114054,0.525431,1.000000,0.059640,0.445487,0.222128,0.350540
Procedures,0.166064,0.036848,0.059640,1.000000,0.005590,-0.039285,0.043749
JCAHO,0.191448,0.434245,0.445487,0.005590,1.000000,0.268122,0.062666
RecordKepping,0.047917,0.169449,0.222128,-0.039285,0.268122,1.000000,0.004203
age,0.113401,0.109200,0.350540,0.043749,0.062666,0.004203,1.000000


In [448]:
joined.dropna(how='any', inplace=True)
joined = joined.query('State != "nan"')
joined.shape

(2813, 15)

# Basic model

In [449]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [450]:
# drop rows with any NaN values
# not a long-term solution, but a quick and dirty model
no_suit = joined[joined.indem_label == 0]
suit_no_indem = joined[joined.indem_label == 1]
suit_indem = joined[joined.indem_label == 2]

no_suit.dropna(how='any', inplace=True)
suit_no_indem.dropna(how='any', inplace=True)
suit_indem.dropna(how='any', inplace=True)

# print totals after NaN drop
print no_suit.shape[0], suit_indem.shape[0], suit_no_indem.shape[0]

# !
# change values to 1 for binary classification
# suit_indem['indem_label'] = 1

# random sample from the three categories
no_suit = no_suit.sample(600)
suit_no_indem = suit_no_indem.sample(246)
suit_indem = suit_indem.sample(34)

2533 34 246


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [451]:
# combine random sample dataframes into concat_df
concat_df = pd.concat([no_suit, suit_indem, suit_no_indem], axis=0)

In [452]:
# build label and feature dataframes
y = concat_df.pop('indem_label')
# ditched: BoardCredit, RecordKepping, state
to_model = concat_df[['Procedures','JCAHO','age','BoardCredit']]
# state = concat_df.pop('State')
# dummies = pd.get_dummies(state)
# to_model = pd.concat([to_model,dummies],axis=1)

In [453]:
# train / test split with random state
X_train, X_test, y_train, y_test = train_test_split(to_model, y, test_size=0.25, stratify=y)

In [454]:
model = RandomForestClassifier(n_estimators=32, max_depth=5 ,oob_score=True)

In [455]:
# fit, validate with OOB score
model.fit(X_train,y_train)
model.oob_score_

0.70757575757575752

In [456]:
preds = model.predict(X_test)
model.score(X_test, y_test)

0.70454545454545459

In [457]:
confusion_matrix(y_test, preds)

array([[133,  17,   0],
       [ 40,  22,   0],
       [  5,   3,   0]])

In [458]:
# grab indices from return_low_no_suit_score FUNC
low_no_suit_scores = return_low_no_suit_score(model,X_test)
indices = [x[0] for x in low_no_suit_scores]

# make y_test a list of values, from DataFrame
y_actual_list = y_test.values.tolist()
# take indices from return_low_no_suit_score FUNC
y_actual_list_appropriate_indices = [y_actual_list[x] for x in indices]

# zip(low_no_suit_scores,y_actual_list_appropriate_indices)

In [459]:
return_probability_scores(model, X_test)

[[0, 0.6076, 0.3405, 0.0519],
 [1, 0.8734, 0.1015, 0.0251],
 [2, 0.7027, 0.2079, 0.0894],
 [3, 0.9333, 0.0615, 0.0052],
 [4, 0.7508, 0.2117, 0.0376],
 [5, 0.6368, 0.3241, 0.0391],
 [6, 0.7261, 0.2514, 0.0225],
 [7, 0.4628, 0.5109, 0.0264],
 [8, 0.6535, 0.3105, 0.036],
 [9, 0.9216, 0.0687, 0.0097],
 [10, 0.5042, 0.4279, 0.0679],
 [11, 0.5458, 0.4005, 0.0537],
 [12, 0.1555, 0.7733, 0.0712],
 [13, 0.7354, 0.2348, 0.0298],
 [14, 0.9333, 0.0615, 0.0052],
 [15, 0.5689, 0.4016, 0.0295],
 [16, 0.9832, 0.0166, 0.0002],
 [17, 0.5042, 0.4279, 0.0679],
 [18, 0.365, 0.6138, 0.0212],
 [19, 0.865, 0.1221, 0.0128],
 [20, 0.4553, 0.4814, 0.0633],
 [21, 0.9832, 0.0166, 0.0002],
 [22, 0.5431, 0.4258, 0.0311],
 [23, 0.7145, 0.2743, 0.0112],
 [24, 0.8816, 0.111, 0.0074],
 [25, 0.8169, 0.1718, 0.0113],
 [26, 0.3988, 0.5407, 0.0605],
 [27, 0.7274, 0.24, 0.0325],
 [28, 0.7261, 0.2514, 0.0225],
 [29, 0.7354, 0.2348, 0.0298],
 [30, 0.3988, 0.5407, 0.0605],
 [31, 0.1633, 0.8084, 0.0283],
 [32, 0.4019, 0.5363, 0.

#### Gridsearching! 

In [460]:
n_est_range = [16,32,64,80,85,90,95,100,120]
m_range = [2,3,4,5,6,8,16,32]
lrgs = GridSearchCV(estimator=model, param_grid=dict(n_estimators=n_est_range, max_depth=m_range), n_jobs=1)
lrgs.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=32, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [16, 32, 64, 80, 85, 90, 95, 100, 120], 'max_depth': [2, 3, 4, 5, 6, 8, 16, 32]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [461]:
print lrgs.best_score_
print lrgs.best_estimator_

0.712121212121
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=95, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)


In [39]:
joined.dropna(how='any', inplace=True)

In [40]:
joined['indem_label'].value_counts()

0.0    3765
1.0     272
2.0      39
Name: indem_label, dtype: int64

In [50]:
sample = joined[joined['indem_label']==2]

In [53]:
sample = sample.append(joined[joined['indem_label']==1].sample(100,replace=False))
sample = sample.append(joined[joined['indem_label']==0].sample(1000,replace=False))

In [55]:
sample.to_csv('sample_of_docs')

In [48]:
sample.indem_label.value_counts()

2.0    39
Name: indem_label, dtype: int64